In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [2]:
from preprocessing import *
data_path = "data/game.csv"
X_train, X_test, y_train, y_test = prep_all(data_path)
# y_test.drop()

In [3]:
X_train = X_train.values.reshape(-1, 1, 135)  # Reshape to (32520, 1, 136)
X_test = X_test.values.reshape(-1, 1, 135)    # Reshape to (N, 1, 136), where N is the number of test samples


In [4]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.svm import SVC
# from sklearn.neural_network import MLPClassifier

# # Define your models and their hyperparameters
# models = [
#     ('Logistic Regression', LogisticRegression(), {'C': [0.1, 1, 10]}),
#     ('Random Forest', RandomForestClassifier(), {'n_estimators': [10, 50, 100], 'max_depth': [None, 10, 20]}),
#     ('SVM', SVC(), {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}),
#     ('Neural Network', MLPClassifier(), {'hidden_layer_sizes': [(10,), (50,), (100,)], 'activation': ['relu', 'tanh']})
# ]

# # Use cross-validation and grid search to compare the models
# for name, model, params in models:
#     grid_search = GridSearchCV(model, params, cv=5, scoring='accuracy')
#     grid_search.fit(X_train, y_train)
    
#     print(f"{name} Best Parameters: {grid_search.best_params_}")
#     print(f"{name} Best Score: {grid_search.best_score_}")


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

In [6]:
train_data = TensorDataset(torch.tensor(X_train), torch.tensor(y_train.to_numpy()))
test_data = TensorDataset(torch.tensor(X_test), torch.tensor(y_test.to_numpy()))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(test_data, batch_size=64, shuffle=False)


In [7]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Set initial hidden and cell states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)

        # Forward propagate RNN
        out, _ = self.rnn(x, h0)

        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        out = nn.functional.softmax(out, dim=1)
        return out


In [8]:
input_size = 135
output_size = 2
device = torch.device("cpu")
hidden_size = 32
num_layers = 4
learning_rate = 0.001
num_epochs = 100
model = RNN(input_size, hidden_size, output_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

def train(model, train_loader, optimizer, criterion, num_epochs=10):
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.float().to(device), labels.long().to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

train(model, train_loader, optimizer, criterion, num_epochs)


Epoch [1/100], Loss: 0.3134
Epoch [2/100], Loss: 0.3133
Epoch [3/100], Loss: 0.3133
Epoch [4/100], Loss: 0.3133
Epoch [5/100], Loss: 0.3133
Epoch [6/100], Loss: 0.3133
Epoch [7/100], Loss: 0.3133
Epoch [8/100], Loss: 0.3133
Epoch [9/100], Loss: 0.3133
Epoch [10/100], Loss: 0.3133
Epoch [11/100], Loss: 0.3133
Epoch [12/100], Loss: 0.3133
Epoch [13/100], Loss: 0.3133
Epoch [14/100], Loss: 0.3133
Epoch [15/100], Loss: 0.3133
Epoch [16/100], Loss: 0.3133
Epoch [17/100], Loss: 0.3133
Epoch [18/100], Loss: 0.3133
Epoch [19/100], Loss: 0.3133
Epoch [20/100], Loss: 0.3133
Epoch [21/100], Loss: 0.3133
Epoch [22/100], Loss: 0.3133
Epoch [23/100], Loss: 0.3133
Epoch [24/100], Loss: 0.3133
Epoch [25/100], Loss: 0.3133
Epoch [26/100], Loss: 0.3133
Epoch [27/100], Loss: 0.3133
Epoch [28/100], Loss: 0.3133
Epoch [29/100], Loss: 0.3133
Epoch [30/100], Loss: 0.3133
Epoch [31/100], Loss: 0.3133
Epoch [32/100], Loss: 0.3133
Epoch [33/100], Loss: 0.3133
Epoch [34/100], Loss: 0.3133
Epoch [35/100], Loss: 0

KeyboardInterrupt: 

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in valid_loader:
        inputs = inputs
        labels = labels.view(-1)
        outputs = model(inputs)
        print(outputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    val_acc = correct / total
    print(f"Validation Accuracy: {val_acc:.4f}")

RuntimeError: mat1 and mat2 must have the same dtype

In [ ]:
def train_model(model, train_loader, criterion, optimizer, device):
    model.train()
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device).view(-1)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

def evaluate_model(model, valid_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in valid_loader:
            inputs, targets = inputs.to(device), targets.to(device).view(-1)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    accuracy = correct / total
    return accuracy

def objective(trial):
    # Define hyperparameters
    hidden_size = trial.suggest_int("hidden_size", 16, 512)
    num_layers = trial.suggest_int("num_layers", 1, 3)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)

    # Create the RNN model
    model = RNN(input_size, hidden_size, output_size, num_layers).to(device)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    num_epochs = 50
    for epoch in range(num_epochs):
        train_model(model, train_loader, criterion, optimizer, device)
        accuracy = evaluate_model(model, valid_loader, device)

        trial.report(accuracy, epoch)

        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy


In [ ]:
# Specify the number of trials
num_trials = 2000
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=num_trials)

# Print the best trial
best_trial = study.best_trial
print(f"Best trial: {best_trial.number}, accuracy: {best_trial.value}")
print(f"Best hyperparameters: {best_trial.params}")


[I 2023-04-30 15:32:37,412] A new study created in memory with name: no-name-b1613d2a-0737-4b56-9414-97081497e47e
/var/folders/29/bwn4vhms0yd66vmmfws1d2800000gn/T/ipykernel_49249/4200047653.py:31: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 1e-2)
[W 2023-04-30 15:33:20,218] Trial 0 failed with parameters: {'hidden_size': 133, 'num_layers': 3, 'learning_rate': 1.1458030014841735e-05} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/spencer/opt/anaconda3/envs/pytorch/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/29/bwn4vhms0yd66vmmfws1d2800000gn/T/ipykernel_49249/4200047653.py", line 43, in objective
    train_

KeyboardInterrupt: 